# Calsificación de estrellas de Hotel con la data hotels.parquet de pyspark

In [1]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=22653d5bcac9358840a64fb029f9c549629cced4a4e26d928f9ff0228a3651ee
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
import pyspark
import requests
import pandas as pd
import tqdm
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import dateutil.parser

In [4]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

spark.conf.set('spark.sql.repl.eagerEval.enabled',True)

In [ ]:
df = spark.read.parquet(f'hotels.parquet', engine='pyarrow')
df

address,checkin_max,checkin_min,checkout_max,checkout_min,description,distance_center,hotel_name,id_city,id_country,id_hotel,lat,lon,price,stars,type,url,url_booking,vote_booking,terraza,ciclismo,senderismo,menús para dietas especiales (bajo petición),bar,restaurante,almuerzos para llevar,proporciona factura,extintores,zona de fumadores,calefacción,ascensor,habitaciones sin humo,catalán,español,portugués,terraza / solárium,jardín,tours en bici,tours a pie,playa,equitación,piragüismo,pesca,cafetería en el alojamiento,snack-bar,servicio de limpieza diario,zona tv / salón de uso compartido,servicio de conserjería,guardaequipaje,fax / fotocopiadora,información turística,registro de entrada y salida exprés,servicio de lavandería,recepción 24 horas,detectores de humo,caja fuerte,prohibido fumar en todo el alojamiento,inglés,mobiliario exterior,barbacoa,vino / champán,cajero automático en el hotel,servicio de habitaciones,cámaras de seguridad fuera del alojamiento,solo para adultos,supermercado pequeño en el alojamiento,aire acondicionado,capilla,de temporada,taquillas en el gimnasio / spa,gimnasio,masaje corporal completo,zona de spa / relajación,baño de vapor,instalaciones de spa,sombrillas,tumbonas/sillas de playa,hammam,bañera de hidromasaje / jacuzzi,masajes,spa y centro de bienestar,alemán,cocina compartida,parking en la calle,registro de entrada / salida privado,plancha para pantalones,centro de negocios,salas de reuniones / banquetes,cámaras de seguridad en las zonas comunitarias,seguridad 24 horas,habitaciones familiares,menú para niños,traslado aeropuerto (de pago),sauna,francés,italiano,alarma de seguridad,habitaciones insonorizadas,para todas las edades,piscina en la azotea,paquetes de spa / bienestar,zona de pícnic,situado frente a la playa,snorkel,windsurf,servicio de planchado,equipamiento de juegos para exterior,adaptado personas de movilidad reducida,música / espectáculos en directo,personal de animación,zona de juegos para niños,sala de juegos,zona de juegos interior,solárium,juegos de mesa / puzles,alquiler de coches,peluquería / salón de belleza,piscina infantil,exfoliación corporal,tratamientos corporales,peinado,tinte,corte de pelo,pedicura,manicura,tratamientos capilares,servicios de maquillaje,servicios de depilación,tratamientos faciales,servicios de belleza,toallas de playa/piscina,campo de golf (a menos de 3 km),submarinismo,pista de tenis,tiro con arco,bolos,billar,esquí,parking en un garaje,servicio de traslado al aeropuerto,servicio de recogida en el aeropuerto,traslado aeropuerto,acceso directo a pistas,escuela de esquí,guardaesquíes,canguro / servicios para niños,piscina de agua salada,clases de yoga,buffet para niños,desayuno en la habitación,parking adaptado para personas de movilidad reducida,servicio de limpieza en seco,máquina expendedora (aperitivos),máquina expendedora (bebidas),guiado auditivo,apoyo visual: carteles táctiles,apoyo visual: braille,cuerda de emergencia en el baño,lavamanos más bajo,wc elevado,wc con barras de apoyo,adaptado para sillas de ruedas,discoteca / dj,dardos,boles para mascotas,fruta,bar en la piscina,zona poco profunda,venta de forfaits,alquiler de equipamiento de esquí en el alojamiento,eventos deportivos en directo (emisión),tour o clase sobre cultura local,ruta de bares,galerías de arte temporales,estación de carga de vehículos eléctricos,masaje de manos,masaje de cabeza,masaje de pies,masaje de cuello,masaje de espalda,habitaciones hipoalergénicas,equipamiento de tenis,club infantil,servicio de entrega de comestibles,servicio de traslado (de pago),abierta todo el año,horario de apertura,piscina climatizada,piscina infinita,piscina con vistas,entrenador personal,clases de fitness,cenas temáticas,baño público,cesta para mascotas,happy hour,entretenimiento nocturno,taquillas,barandillas protectoras para bebés,piscina pequeña,baño al aire libre,valla alrededor de la piscina,holandés,gallego,ping pong,aeróbic,cambio de moneda,pediluvio,minigolf,wifi de pago,polaco,instal

In [ ]:
df.count()

353317

In [ ]:
df.printSchema()

root
 |-- address: string (nullable = true)
 |-- checkin_max: string (nullable = true)
 |-- checkin_min: string (nullable = true)
 |-- checkout_max: string (nullable = true)
 |-- checkout_min: string (nullable = true)
 |-- description: string (nullable = true)
 |-- distance_center: double (nullable = true)
 |-- hotel_name: string (nullable = true)
 |-- id_city: long (nullable = true)
 |-- id_country: long (nullable = true)
 |-- id_hotel: long (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- price: double (nullable = true)
 |-- stars: long (nullable = true)
 |-- type: long (nullable = true)
 |-- url: string (nullable = true)
 |-- url_booking: string (nullable = true)
 |-- vote_booking: double (nullable = true)
 |-- terraza: long (nullable = true)
 |-- ciclismo: long (nullable = true)
 |-- senderismo: long (nullable = true)
 |-- menús para dietas especiales (bajo petición): long (nullable = true)
 |-- bar: long (nullable = true)
 |-- restaurant

# Conseguir el numero de hoteles de cada categoria de estrellas por pais

In [ ]:
df.groupBy('id_country','stars').agg(f.count('*').alias('numero_hoteles')).sort(f.desc('numero_hoteles')).show()

+----------+-----+--------------+
|id_country|stars|numero_hoteles|
+----------+-----+--------------+
|        98|    3|         11614|
|        24|    3|         10577|
|       220|    3|          8831|
|       105|    3|          8346|
|       105|    0|          7993|
|       195|    3|          7942|
|       195|    4|          6744|
|       167|    0|          6386|
|        75|    3|          5957|
|        33|    3|          5708|
|        33|    0|          5410|
|       219|    0|          5299|
|        24|    0|          5036|
|        24|   -1|          4933|
|        81|    0|          4557|
|       105|    4|          4541|
|       195|    0|          4477|
|        14|    0|          4452|
|        75|    0|          3821|
|       107|    3|          3797|
+----------+-----+--------------+
only showing top 20 rows



# Extraer el nombre del pais en una nueva columna

In [ ]:
df=df.withColumn('country', f.element_at(f.split(f.col('address'),','),-1))
df.select('id_country','country').distinct().show(155000)
df.select('id_country','country').distinct().count()

+----------+--------------------+
|id_country|             country|
+----------+--------------------+
|        30|             Bolivia|
|        70|             Etiopía|
|        81|            Alemania|
|        26|              Belice|
|       167|             Polonia|
|        66|         El Salvador|
|        50|               Congo|
|       211|               Túnez|
|       230|              Zambia|
|       191|             Somalia|
|       207|      Timor Oriental|
|        13|   Antigua y Barbuda|
|       165|                Perú|
|       196|           Sri Lanka|
|       204|         Tadjikistán|
|       101|                Irak|
|       130|              Malawi|
|        24|               Rusia|
|       150|               Nepal|
|        46|               Chile|
|       206|           Tailandia|
|       164|            Paraguay|
|        59|     Costa de Marfil|
|        39|          Cabo Verde|
|       183|             Senegal|
|       118|                Laos|
|        32|  

184

# Extraer el nombre de la ciudad en una nueva columna

In [ ]:
df=df.withColumn('city', f.element_at(f.split(f.col('address'),','),-2))
df.select('id_city','city').distinct().show(5000)
df.select('id_city','city').distinct().count()

df=df.withColumn('city', f.regexp_replace(f.element_at(f.split(f.col('address'),','),-2),'\d',''))
df.filter(f.col('id_country')==198).select('city').distinct()

+-------+------------------------------------+
|id_city|                                city|
+-------+------------------------------------+
|  35151|                      100097 Beijing|
|  33907|                       75080 Karachi|
|  22863|                               Tuzla|
|  29400|                        119421 Moscú|
|  29400|                        117208 Moscú|
|  10617|                       CEP 01151-000|
|  10617|                       CEP 01404-000|
|  10617|                       CEP 04018-033|
|  10617|                       CEP 01508-000|
|  10617|                       CEP 02113-012|
|  10617|                       CEP 01453-020|
|  10617|                       CEP 01451-010|
|   5399|                        110040 Delhi|
|  10186|                      211100 Nanjing|
|  33615|                      12651 El Cairo|
|   6730|                 20908 Ciudad de ...|
|   6730|                 01110 Ciudad de ...|
|   6730|                 52754 San Pedro ...|
|   6730|    

city
Kwatta Paramaribo
Paramaribo
Paramaribo
**** Paramaribo
Paramaribo
AA Paramaribo


# Extraer el numero de hoteles por cada pais que se encuentren en playa

In [ ]:
#df=df.groupBy('country','playa').agg(f.count('*').alias('en_playa')).sort(f.desc('en_playa'))
#df
df.filter(f.col('playa')==1).groupBy('country').agg(f.count('*').alias('en_playa')).sort(f.desc('en_playa'))#.select('country','en_playa')

country,en_playa
Brasil,640
España,520
México,384
Vietnam,328
Reino Unido,250
Estados Unidos,238
Rusia,214
Sudáfrica,190
Corea del Sur,190
Turquía,187


# Identificar las caracteristicas importantes que determinen las estrellas del hotel

In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline

# Realizar un modelo de clustering por zona geografica

In [ ]:
vecAssembler = VectorAssembler(inputCols=['lat', 'lon'], outputCol="features")
km = KMeans().setK(3) \
                .setParams(predictionCol='geografical_cluster') \
                .setParams(featuresCol='features')

pipeline = Pipeline(stages=[vecAssembler, km])
model = pipeline.fit(df)

In [ ]:
df_kmeans = model.transform(df)
df_kmeans.select('country','geografical_cluster').distinct().sort(f.desc('geografical_cluster')).show()

+--------------------+-------------------+
|             country|geografical_cluster|
+--------------------+-------------------+
|             Curaçao|                  2|
|              Brasil|                  2|
|               Chile|                  2|
|               Haití|                  2|
|   Antigua y Barbuda|                  2|
|                Perú|                  2|
| San Cristóbal y ...|                  2|
|             Granada|                  2|
|         Puerto Rico|                  2|
|             Bahamas|                  2|
|           Venezuela|                  2|
|              México|                  2|
|               Samoa|                  2|
|         El Salvador|                  2|
|            Bermudas|                  2|
|           Argentina|                  2|
|           Guatemala|                  2|
|      Estados Unidos|                  2|
|             Surinam|                  2|
| República Domini...|                  2|
+----------

# Realizar un modelo de clasificacion para identificar las estrellas del hotel

In [ ]:
features=df.columns[13:-3]
features

In [ ]:
columnas_hechas = ['price',
 'stars',
 'type',
 'url',
 'url_booking',
 'vote_booking',
 'terraza',
 'ciclismo',
 'senderismo',
 'menús para dietas especiales (bajo petición)',
 'bar',
 'restaurante',
 'almuerzos para llevar',
 'proporciona factura',
 'extintores',
 'zona de fumadores',
 'calefacción',
 'ascensor',
 'habitaciones sin humo',
 'catalán',
 'español',
 'portugués',
 'terraza / solárium',
 'jardín',
 'tours en bici',
 'tours a pie',
 'playa',
 'equitación',
 'piragüismo',
 'pesca',
 'cafetería en el alojamiento',
 'snack-bar',
 'servicio de limpieza diario',
 'zona tv / salón de uso compartido',
 'servicio de conserjería',
 'guardaequipaje',
 'fax / fotocopiadora',
 'información turística',
 'registro de entrada y salida exprés',
 'servicio de lavandería',
 'recepción 24 horas',
 'detectores de humo',
 'caja fuerte',
 'prohibido fumar en todo el alojamiento',
 'inglés',
 'mobiliario exterior',
 'barbacoa',
 'vino / champán',
 'cajero automático en el hotel',
 'servicio de habitaciones',
 'cámaras de seguridad fuera del alojamiento',
 'solo para adultos',
 'supermercado pequeño en el alojamiento',
 'aire acondicionado',
 'capilla',
 'de temporada',
 'taquillas en el gimnasio / spa',
 'gimnasio',
 'masaje corporal completo',
 'zona de spa / relajación',
 'baño de vapor',
 'instalaciones de spa',
 'sombrillas',
 'tumbonas/sillas de playa',
 'hammam',
 'bañera de hidromasaje / jacuzzi',
 'masajes',
 'spa y centro de bienestar',
 'alemán',
 'cocina compartida',
 'parking en la calle',
 'registro de entrada / salida privado',
 'plancha para pantalones',
 'centro de negocios',
 'salas de reuniones / banquetes',
 'cámaras de seguridad en las zonas comunitarias',
 'seguridad 24 horas',
 'habitaciones familiares',
 'menú para niños',
 'traslado aeropuerto (de pago)',
 'sauna',
 'francés',
 'italiano',
 'alarma de seguridad',
 'habitaciones insonorizadas',
 'para todas las edades',
 'piscina en la azotea',
 'paquetes de spa / bienestar',
 'zona de pícnic',
 'situado frente a la playa',
 'snorkel',
 'windsurf',
 'servicio de planchado',
 'equipamiento de juegos para exterior',
 'adaptado personas de movilidad reducida',
 'música / espectáculos en directo',
 'personal de animación',
 'zona de juegos para niños',
 'sala de juegos',
 'zona de juegos interior',
 'solárium',
 'juegos de mesa / puzles',
 'alquiler de coches',
 'peluquería / salón de belleza',
 'piscina infantil',
 'exfoliación corporal',
 'tratamientos corporales',
 'peinado',
 'tinte',
 'corte de pelo',
 'pedicura',
 'manicura',
 'tratamientos capilares',
 'servicios de maquillaje',
 'servicios de depilación',
 'tratamientos faciales',
 'servicios de belleza',
 'toallas de playa/piscina',
 'campo de golf (a menos de 3 km)',
 'submarinismo',
 'pista de tenis',
 'tiro con arco',
 'bolos',
 'billar',
 'esquí',
 'parking en un garaje',
 'servicio de traslado al aeropuerto',
 'servicio de recogida en el aeropuerto',
 'traslado aeropuerto',
 'acceso directo a pistas',
 'escuela de esquí',
 'guardaesquíes',
 'canguro / servicios para niños',
 'piscina de agua salada',
 'clases de yoga',
 'buffet para niños',
 'desayuno en la habitación',
 'parking adaptado para personas de movilidad reducida',
 'servicio de limpieza en seco',
 'máquina expendedora (aperitivos)',
 'máquina expendedora (bebidas)',
 'guiado auditivo',
 'apoyo visual: carteles táctiles',
 'apoyo visual: braille',
 'cuerda de emergencia en el baño',
 'lavamanos más bajo',
 'wc elevado',
 'wc con barras de apoyo',
 'adaptado para sillas de ruedas',
 'discoteca / dj',
 'dardos',
 'boles para mascotas',
 'fruta',
 'bar en la piscina',
 'zona poco profunda',
 'venta de forfaits',
 'alquiler de equipamiento de esquí en el alojamiento',
 'eventos deportivos en directo (emisión)',
 'tour o clase sobre cultura local',
 'ruta de bares',
 'galerías de arte temporales',
 'estación de carga de vehículos eléctricos',
 'masaje de manos',
 'masaje de cabeza',
 'masaje de pies',
 'masaje de cuello',
 'masaje de espalda',
 'habitaciones hipoalergénicas',
 'equipamiento de tenis',
 'club infantil',
 'servicio de entrega de comestibles',
 'servicio de traslado (de pago)',
 'abierta todo el año',
 'horario de apertura',
 'piscina climatizada',
 'piscina infinita',
 'piscina con vistas',
 'entrenador personal',
 'clases de fitness',
 'cenas temáticas',
 'baño público',
 'cesta para mascotas',
 'happy hour',
 'entretenimiento nocturno',
 'taquillas',
 'barandillas protectoras para bebés',
 'piscina pequeña',
 'baño al aire libre',
 'valla alrededor de la piscina',
 'holandés',
 'gallego',
 'ping pong',
 'aeróbic',
 'cambio de moneda',
 'pediluvio',
 'minigolf',
 'wifi de pago',
 'polaco',
 'instalaciones para deportes acuáticos en el alojamiento',
 'chimenea exterior',
 'equipamiento de bádminton',
 'juguetes para la piscina',
 'alquiler de bicicletas (de pago)',
 'sueco',
 'bicicletas disponibles (gratis)',
 'parking vigilado',
 'cochecitos de bebé',
 'noches de cine',
 'envoltura corporal',
 'bingo',
 'árabe',
 'tarjetas de transporte público',
 'clases de cocina',
 'servicio de traslado',
 'servicio de traslado (gratis)',
 'masaje en pareja',
 'euskera',
 'comedia en vivo',
 'parque acuático',
 'silla de masajes',
 'rumano',
 'squash',
 'danés',
 'noruego',
 'terapia con luz',
 'baños termales',
 'traslado aeropuerto (gratis)',
 'ruso',
 'papel higiénico',
 'toallas',
 'bañera o ducha',
 'baño privado',
 'wc',
 'artículos de aseo gratis',
 'secador de pelo',
 'ducha',
 'ropa de cama',
 'armario',
 'tostadora',
 'fogones',
 'utensilios de cocina',
 'cocina',
 'microondas',
 'nevera',
 'zona de cocina',
 'enchufe cerca de la cama',
 'perchero',
 'suelo de madera o parquet',
 'ventilador',
 'sofá',
 'zona de estar',
 'escritorio',
 'tv de pantalla plana',
 'tv',
 'acceso a pisos superiores solo mediante escaleras',
 'apartamento privado en edificio',
 'insonorización',
 'karaoke',
 'bañera',
 'filipino',
 'tobogán de agua',
 'chino',
 'ucraniano',
 'horno',
 'lavadora',
 'aseo compartido',
 'chimenea',
 'tendedero',
 'suelo de baldosa / mármol',
 'entrada privada',
 'plancha para ropa',
 'muebles de exterior',
 'patio',
 'vistas al jardín',
 'independiente',
 'esloveno',
 'cubierta para piscina',
 'bidet',
 'toallas / sábanas (por un suplemento)',
 'vistas a la montaña',
 'canales por cable',
 'canales vía satélite',
 'libros, dvd o música para niños',
 'balcón',
 'mesa de comedor',
 'zona de comedor',
 'acceso al salón executive',
 'servicio de despertador',
 'equipo de planchado',
 'croata',
 'eslovaco',
 'zona privada de playa',
 'lituano',
 'georgiano',
 'productos de limpieza',
 'casino',
 'moldavo',
 'camas extralargas (más de 2 metros)',
 'vistas a un lugar de interés',
 'vistas',
 'teléfono',
 'minibar',
 'servicio de despertador / alarma',
 'acceso a pisos superiores en ascensor',
 'radio',
 'toda la unidad accesible en silla de ruedas',
 'hervidor eléctrico',
 'lavavajillas',
 'sofá cama',
 'vistas a un patio interior',
 'aseo adicional',
 'zona de comedor exterior',
 'cama plegable',
 'búlgaro',
 'checo',
 'trona',
 'hipoalergénico',
 'tetera / cafetera',
 'adecuada para niños',
 'albornoz',
 'vistas al lago',
 'japonés',
 'afrikáans',
 'zapatillas',
 'vistas al mar',
 'vietnamita',
 'húngaro',
 'vestidor',
 'caja fuerte para ordenador portátil',
 'finlandés',
 'persa',
 'wifi',
 'canales de pago',
 'mosquitera',
 'indonesio',
 'hebreo',
 'bañera de hidromasaje',
 'reloj despertador',
 'vistas a la piscina',
 'urdú',
 'serbio',
 'secadora de ropa',
 'griego',
 'coreano',
 'ordenador portátil',
 'ordenador',
 'reproductor de cd',
 'reproductor de dvd',
 'barandillas de seguridad para bebés',
 'turco',
 'hindi',
 'letón',
 'suelo de moqueta',
 'piscina privada',
 'galés',
 'suajili',
 'toda la unidad en la planta baja',
 'armenio',
 'protector de enchufes para niños',
 'fax',
 'habitaciones comunicadas disponibles',
 'baño adicional',
 'albanés',
 'baño compartido',
 'detector de monóxido de carbono',
 'soporte para ipod',
 'bosnio',
 'vídeo',
 'estonio',
 'ipad',
 'videojuegos',
 'mantas eléctricas',
 'malayo',
 'cantonés',
 'videoconsola nintendo wii',
 'adosado',
 'bereber',
 'videoconsola ps2',
 'internet',
 'tarjeta de acceso',
 'llave de acceso',
 'videoconsola ps3',
 'panyabí',
 'zulú',
 'macedonio',
 'azerí',
 'irlandés',
 'bengalí',
 'mandarín',
 'tatami (suelo tradicional japonés)',
 'maratí',
 'safari en coche',
 'canarés',
 'malabar',
 'tamil',
 'télugu',
 'montenegrino',
 'xhosa',
 'guyaratí',
 'maltés',
 'feroés',
 'islandés',
 'tailandés',
 'oriya',
 'jemer',
 'yoruba',
 'hausa',
 'guaraní',
 'laosiano',
 'bañera al aire libre',
 'mongol',
 'birmano',
 'groenlandés',
 'bielorruso',
 'javanés',
 'baños públicos',
 'bath/hot spring']

In [ ]:
#Separar en training - test los datos *
train_df, test_df = df.randomSplit([0.7, 0.3])
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StandardScaler
#continuous_feature_scaler = StandardScaler(inputCol="unscaled_continuous_features", outputCol="scaled_continuous_features", withStd=True, withMean=False)

In [ ]:
lr = LinearRegression( labelCol='stars', predictionCol='stars_prediction')
featurePipeline = Pipeline(stages=[pipeline] +  [lr])
model = featurePipeline.fit(df)

In [ ]:
result_df = model.transform(df)
result_df.select('stars','stars_prediction').distinct().sort(f.desc('stars_prediction')).show()

+-----+------------------+
|stars|  stars_prediction|
+-----+------------------+
|    0|1.9754132413385763|
|    3|1.9752964218249809|
|    3|1.9752947601198318|
|    3| 1.975292826498363|
|    3|1.9752888052357265|
|    3|1.9752884891478373|
|    3|1.9752881951729047|
|    0|1.9752844695301248|
|    0|1.9752830614134085|
|    0| 1.975282928926845|
|    3|1.9752818396395262|
|    3|1.9752817846198247|
|   -1|1.9752792879759498|
|    3| 1.975279041398308|
|    3|1.9752784277011013|
|    0| 1.975277237801946|
|    3|1.9752764089013373|
|    3|1.9752763586436042|
|    0| 1.975276299373903|
|    0|1.9752724211325554|
+-----+------------------+
only showing top 20 rows

